# Problem Statement

Define:
- [Time] $t \in [0, T] \subset \mathbb{R}_+$ be time.
- [State] $x(t;\cdot), u(t;\cdot)$ be state variables (resp. continuous, non-differentiable) where $x(t) \in X\subset \mathbb{R}^{n_x}, u(t)\in U \subset\mathbb{R}^{n_u}$.
- [Parameters] $p, q$ be a set of design parameters with $p\in P \subset \mathbb{R}^{n_p}, q \in Q \subset\mathbb{R}^{n_q}$
- [Dynamics] $f: X \times U \times P \times Q \rightarrow TX \subset\mathbb{R}^{n_x}$ be a smooth vector field.
- [Control] $g: X \times P \times Q \rightarrow Y$ be a function, not-necessarily differentiable but computationally 'cheap' to evaluate.
- [Constraints] $h: X \times U \times P\times Q \rightarrow \mathbb{R}^{n_c}$
- [Loss] $\phi: X \times U \times P \times Q \rightarrow \mathbb{R}_+ $ be a continuous function bounded below (w.log by $0$).

We use the following shorthand: $x = x(t;p,q)$ (similarly with $u$) $x(T) = x(T;p,q)$ and $\phi(p,q) = \phi(x(T), u(T); p,q)$.

### [Problem 1] Dynamic Optimisation

$$
\begin{align}
\min_{p\in P ,q \in Q } &&\phi(x(T), u(T); p,q)&&&\\
\text{s.t.}&
& \dot{x} =& f(x,u; p,q), &&\\
&& u  = & g(x; p, q),&& \\
&& h_i(x,u;p,q) \le& 0 && i \in 1,\ldots, n_c\\
&&x\in&X,&&\\
&&u\in&U.&&
\end{align}
$$

#### Comments
- Integral loss $J = \int_0^T j(x,u;p,q)dt$ or constraints can be incorporated by adding additional state variables $x_J$ such that $\dot{x}_J = j(x,u;p,q)$
- Loss as a function of time $T$ can be similarly handled.

## Computational Expense and Black Box Optimisation

Suppose that one has a candidate pair $(p,q)_0$ that solve problem 1.
One wishes to find a sequence $(p,q)_0,(p,q)_1,\ldots (p,q)_k$ such that $\phi(p_k,q_k) < \phi(p_{k-1},q_{k-1})$

The challenge is as follows:
- Given $p_k,q_k$ and $\phi(p_k,q_k)$, computing $\phi(p_{k + 1}, q_k)$ is much 'cheaper' than computing $\phi(p_k, q_{k+1})$
- The problem is not necessarily differentiable (eg, $g$ is a bang-bang controller), nor convex.



b*Derivative(x(t), t) - l*m*sin(\theta(t))*Derivative(\theta(t), t)**2 + l*m*cos(\theta(t))*Derivative(\theta(t), (t, 2)) - u + (M_c + m)*Derivative(x(t), (t, 2))

-g*l**2*m**2*sin(2*\theta(t))/2 - l*m*(I + l**2*m)*sin(\theta(t))*Derivative(\theta(t), t)**2 + u*(-I - l**2*m) + (I*b + b*l**2*m)*Derivative(x(t), t) + (I*M_c + I*m + M_c*l**2*m + l**2*m**2*sin(\theta(t))**2)*Derivative(x(t), (t, 2))

g*l**2*m**2*sin(2*\theta(t))/2 + u*(I + l**2*m) + (-I*b - b*l**2*m)*Derivative(x(t), t) + (I*l*m*sin(\theta(t)) + l**3*m**2*sin(\theta(t)))*Derivative(\theta(t), t)**2

I*M_c + I*m + M_c*l**2*m + l**2*m**2*sin(\theta(t))**2

(-g*l*m*(I*M_c + I*m + M_c*l**2*m + l**2*m**2*sin(\theta(t))**2)*sin(\theta(t)) - l*m*(g*l**2*m**2*sin(2*\theta(t))/2 + u*(I + l**2*m) + (-I*b - b*l**2*m)*Derivative(x(t), t) + (I*l*m*sin(\theta(t)) + l**3*m**2*sin(\theta(t)))*Derivative(\theta(t), t)**2)*cos(\theta(t)))/(I + l**2*m)

In [6]:
test_subs = [
    (ddtheta, f_dtheta / r),
    (ddx,  f_dx / r)
]
residue = [eqn.subs(test_subs) for eqn in dynamics]
residue = [(r * r_i).expand().simplify() for r_i in residue]
residue
# if result is (0,0) -> equations are correct

[0, 0]

In [7]:
# r[ddx, ddtheta] = [f_dx, f_dtheta]

# Key substitutions
c  = sp.symbols('c:3', positive=True)
subs = [
    (I, c[0] - m * l * l),   # c[0] = I + m l^2
    (l, c[1] / m),           # c[1] = lm
    (m, c[2] - M_c)            # c[2] = M_c + m
]


r = sp.trigsimp(sp.expand(r.subs(subs)))
f_dtheta = sp.simplify(sp.expand_trig(sp.simplify(sp.expand(f_dtheta.subs(subs))))).expand()
f_dx = sp.simplify(sp.expand_trig(sp.expand(f_dx.subs(subs))))

X_old = (dx, dtheta, x, theta)
X_list = sp.symbols('x:4')
subs_X = [(x_i, X_i) for x_i, X_i in zip(X_old,X_list)]
X = sp.Matrix(X_list)
G = sp.Matrix([f_dx.coeff(u)/c[0], f_dtheta.coeff(u)/c[1], 0,0]).subs(subs_X)
F = sp.Matrix([
    sp.simplify(f_dx/c[0] - G[0]*u),
    sp.simplify(f_dtheta/c[1] - G[1]*u),
    dx,
    dtheta
]).subs(subs_X)
M = sp.diag(r/c[0],r/c[1],1,1).subs(subs_X)
Q = sp.eye(4)
R = 1
X_0 = sp.Matrix([0,sp.pi, 0, 0])
phi = sp.Matrix(list(sp.symbols(r'\lambda_:4')))

In [8]:
d_star = [(d.subs(subs)).expand() for d in dynamics]
d_star[0] = d_star[0].subs(ddtheta, (ddtheta -  d_star[1]/c[0]).expand()).expand().collect([ddx,dx,dtheta])
u_d = sp.symbols('u_d')
factor = d_star[0].coeff(ddx)
u_subs = u_d * d_star[0].coeff(ddx) + (d_star[0] - ddx * d_star[0].coeff(ddx) + u)
d_star[0] = (d_star[0].subs(u, u_subs)/factor).cancel()
d_star[1] = (d_star[1].subs(ddx, ddx - d_star[0])/c[0]).expand()

In [9]:
d_star[0]

-u_d + Derivative(x(t), (t, 2))

In [10]:
d_star[1]

Derivative(\theta(t), (t, 2)) + c1*g*sin(\theta(t))/c0 + c1*u_d*cos(\theta(t))/c0

In [18]:
E = c[0] * (dtheta**2)/2 - c[1] * g *(  sp.cos(theta) - 1)
k = sp.symbols("k:3")
u_ff = k[0]*dtheta*sp.cos(theta) * E - k[1]*x - k[2]*dx

In [25]:
u_final = u_subs.subs(u_d, u_ff).simplify()
ux = u_final.subs(subs_X)
ux.expand()

b*x0 + c0*c2*k0*x1**3*cos(x3)/2 - c1**2*k0*x1**3*cos(x3)**3/2 - c1*c2*g*k0*x1*cos(x3)**2 + c1*c2*g*k0*x1*cos(x3) - c1*x1**2*sin(x3) - c2*k1*x2 - c2*k2*x0 + c1**3*g*k0*x1*cos(x3)**4/c0 - c1**3*g*k0*x1*cos(x3)**3/c0 - c1**2*g*sin(2*x3)/(2*c0) + c1**2*k1*x2*cos(x3)**2/c0 + c1**2*k2*x0*cos(x3)**2/c0

In [20]:
F

Matrix([
[-b*x0 + c1*x1**2*sin(x3) + c1**2*g*sin(2*x3)/(2*c0)],
[ b*x0*cos(x3) - c1*x1**2*sin(2*x3)/2 - c2*g*sin(x3)],
[                                                 x0],
[                                                 x1]])

Matrix([
[       1],
[-cos(x3)],
[       0],
[       0]])